In [ ]:

!pip install --upgrade --no-cache-dir gdown
!pip install rembg[gpu]

In [ ]:

! wget -c "https://github.com/Kitware/CMake/releases/download/v3.19.6/cmake-3.19.6.tar.gz"
! tar xf cmake-3.19.6.tar.gz
! cd cmake-3.19.6 && ./configure && make && sudo make install
     

In [ ]:

# Install library
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

In [ ]:

ver_openpose = "v1.7.0"
! echo $ver_openpose

In [ ]:

# manually downloading openpose models
%%bash
gdown 1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh
unzip models.zip
mv /content/models/face/pose_iter_116000.caffemodel /content/openpose/models/face/pose_iter_116000.caffemodel
mv /content/models/hand/pose_iter_102000.caffemodel /content/openpose/models/hand/pose_iter_102000.caffemodel
mv /content/models/pose/body_25/pose_iter_584000.caffemodel /content/openpose/models/pose/body_25/pose_iter_584000.caffemodel
mv /content/models/pose/coco/pose_iter_440000.caffemodel /content/openpose/models/pose/coco/pose_iter_440000.caffemodel
mv /content/models/pose/mpi/pose_iter_160000.caffemodel /content/openpose/models/pose/mpi/pose_iter_160000.caffemodel
rm -rf models
rm models.zip

In [ ]:

! cd openpose && mkdir build && cd build

In [ ]:
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git

In [ ]:

! cd openpose/build && cmake -DUSE_CUDNN=OFF -DBUILD_PYTHON=ON 

In [ ]:

! cd openpose/build && make -j`nproc`
! cd openpose && mkdir output

In [ ]:

!pip install flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 

In [ ]:

import os
%cd /content/
!rm -rf clothes-virtual-try-on
!git clone https://github.com/practice404/clothes-virtual-try-on.git
os.makedirs("/content/clothes-virtual-try-on/checkpoints")
     

In [ ]:
!gdown --id 18q4lS7cNt1_X8ewCgya1fq0dSk93jTL6 --output /content/clothes-virtual-try-on/checkpoints/alias_final.pth
!gdown --id 1uDRPY8gh9sHb3UDonq6ZrINqDOd7pmTz --output /content/clothes-virtual-try-on/checkpoints/gmm_final.pth
!gdown --id 1d7lZNLh51Qt5Mi1lXqyi6Asb2ncLrEdC --output /content/clothes-virtual-try-on/checkpoints/seg_final.pth
     

In [ ]:
!gdown --id 1ysEoAJNxou7RNuT9iKOxRhjVRNY5RLjx --output /content/clothes-virtual-try-on/cloth_segm_u2net_latest.pth --no-cookies
     

In [ ]:

%cd /content/
!pip install ninja
     

In [ ]:

!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing
!mkdir checkpoints
     

In [ ]:

%cd /content/

In [ ]:

%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/
MINICONDA_INSTALLER_SCRIPT
./
MINICONDA_PREFIX
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

In [ ]:


import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))
     

In [ ]:

!pip install torchvision
     

In [ ]:
def make_dir():
  os.system("cd /content/ && mkdir inputs && cd inputs && mkdir test && cd test && mkdir cloth cloth-mask image image-parse openpose-img openpose-json")
     

In [ ]:

from flask import Flask, request, send_file, jsonify
from flask_ngrok import run_with_ngrok
from PIL import Image
import base64
import io

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
  return jsonify("hello world");

@app.route("/api/transform", methods=['POST'])
def begin():
  make_dir()
  print("data recieved")
  cloth = request.files['cloth']
  model = request.files['model']

  cloth = Image.open(cloth.stream)
  model = Image.open(model.stream)

  cloth.save("/content/inputs/test/cloth/cloth.jpg")
  model.save("/content/inputs/test/image/model.jpg")

  # running script to compute the predictions
  os.system("python /content/clothes-virtual-try-on/run.py")

  # loading output
  op = os.listdir("/content/output")[0]
  op = Image.open(f"/content/output/{op}")
  buffer = io.BytesIO()
  op.save(buffer, 'png')
  buffer.seek(0)
  os.system("rm -rf /content/output/")
  return send_file(buffer, mimetype='image/gif')

In [ ]:

if __name__ == '__main__':
    app.run()

In [ ]:

!conda install --channel conda-forge featuretools --yes

In [ ]:

!pip install opencv-python torchgeometry